In [1]:
#default_exp sentryUtil

# sentryUtil

In [2]:
#hide
import sentry_sdk
sentry_sdk.init(
    "https://0bab4064545e4a95b35d4e73f20f7632@o839457.ingest.sentry.io/5894971",

    # Set traces_sample_rate to 1.0 to capture 100%
    # of transactions for performance monitoring.
    # We recommend adjusting this value in production.
    traces_sample_rate=1.0
)

In [3]:
#export
from sentry_sdk import add_breadcrumb, capture_exception, capture_message
from nicHelper.exception import traceback
from typing import Any

In [4]:
#export
def logSentry(message:str, data:Any = (lambda :{})(), level:str = 'info', section:str='main'):
  '''
    just add docs for ease of logging to sentry
    Input:
      message ::str:: required :: message to send to sentry
      data ::dict:: optional :: and object to send to sentry (default is an empty dict)
      level ::str::optional:: log level (default:info)
      section ::str::optional:: section name or function name (default: main)
    Response:
      Bool:: true means logged properly, false for error, print error message to console
  '''
  try:
    add_breadcrumb(
      category=section,
      data={'data':data},
      level=level,
      message=message
    )
    return True
  except Exception as e:
    print(message, data, level, section)
    print(f'error is {e}, {traceback()}')
    return False

In [5]:
logSentry('hello', {'hello':'this is a test'}, level='error', section = 'test')
capture_message('testing')

'dc12ff9a1b9d421fbf03391b6b75114c'

## full example

In [6]:
from nicHelper.sentryUtil import logSentry
logSentry('this is a test', {'testobject': 'testvalue'})

True